In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install ultralytics --quiet
!pip install optuna --quiet

import torch
from ultralytics import YOLO
import optuna
import os
import yaml

dataset_config = {
    "train": "/kaggle/input/mars-daily-images/dataset/images/train",
    "val": "/kaggle/input/mars-daily-images/dataset/images/val",
    "nc": 1,
    "names": ["dust_storm"]
}

dataset_yaml_path = "/kaggle/working/dataset_fixed.yaml"
with open(dataset_yaml_path, "w") as f:
    yaml.dump(dataset_config, f)

print("CUDA Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

def objective(trial):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
    mosaic = trial.suggest_float("mosaic", 0.4, 0.8)
    mixup = trial.suggest_float("mixup", 0.0, 0.2)
    scale = trial.suggest_float("scale", 0.2, 0.4)
    translate = trial.suggest_float("translate", 0.05, 0.15)
    fliplr = trial.suggest_float("fliplr", 0.3, 0.7)

    model = YOLO('yolov8m.pt')
    
    results = model.train(
        data=dataset_yaml_path,
        epochs=20,
        batch=8,
        imgsz=960,
        device=device,
        workers=4,
        hsv_h=0.015,
        hsv_s=0.25,
        hsv_v=0.2,
        translate=translate,
        scale=scale,
        perspective=0.0,
        flipud=0.0,
        fliplr=fliplr,
        mosaic=mosaic,
        close_mosaic=10,
        mixup=mixup,
        erasing=0.0,
        auto_augment=None,
        optimizer="AdamW",
        lr0=lr0,
        lrf=0.01,
        cos_lr=True,
        warmup_epochs=3,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        weight_decay=0.0005,
        dropout=0.05,
        freeze=0,
        patience=10,
        amp=True,
        seed=42,
        deterministic=True,
        val=True,
        plots=False,
        multi_scale=False,
        project="/kaggle/working/runs/detect_optuna",
        name=f"trial_{trial.number}",
        save_period=1,
        save=True
    )

    try:
        best_map50 = results[0].metrics.get("mAP50", 0.0)
    except Exception:
        best_map50 = 0.0

    print(f"Trial {trial.number} | lr0: {lr0:.6f}, mosaic: {mosaic:.3f}, mixup: {mixup:.3f}, scale: {scale:.3f}, translate: {translate:.3f}, fliplr: {fliplr:.3f} => mAP50: {best_map50:.3f}")
    return best_map50

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10, timeout=3600)

print("Best hyperparameters found:")
print(study.best_trial.params)
print("Best mAP50:", study.best_value)

best_params = study.best_trial.params
with open("/kaggle/working/best_hyperparams.yaml", "w") as f:
    yaml.dump(best_params, f)

def full_training():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Full training on: {device.upper()}")

    with open("/kaggle/working/best_hyperparams.yaml", "r") as f:
        best_params = yaml.safe_load(f)

    model = YOLO('yolov8m.pt')

    model.train(
        data=dataset_yaml_path,
        epochs=200,
        batch=8,
        imgsz=960,
        device=device,
        workers=4,
        hsv_h=0.015,
        hsv_s=0.25,
        hsv_v=0.2,
        translate=best_params.get("translate", 0.1),
        scale=best_params.get("scale", 0.3),
        perspective=0.0,
        flipud=0.0,
        fliplr=best_params.get("fliplr", 0.5),
        mosaic=best_params.get("mosaic", 0.6),
        close_mosaic=10,
        mixup=best_params.get("mixup", 0.1),
        erasing=0.0,
        auto_augment=None,
        optimizer="AdamW",
        lr0=best_params.get("lr0", 0.0005),
        lrf=0.01,
        cos_lr=True,
        warmup_epochs=3,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        weight_decay=0.0005,
        dropout=0.05,
        freeze=0,
        patience=30,
        amp=True,
        seed=42,
        deterministic=True,
        val=True,
        plots=True,
        multi_scale=False,
        project="/kaggle/working/runs/detect_full",
        name="train150_yolov8m_auto_hp",
        save_period=1,
        save=True
    )

if __name__ == "__main__":
    pass

In [ ]:
!pip install ultralytics opencv-python tqdm --quiet

import os
import torch
from ultralytics import YOLO

print("CUDA Available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

dataset_yaml_content = """
train: /kaggle/input/mars-daily-images/dataset/images/train
val: /kaggle/input/mars-daily-images/dataset/images/val

nc: 1
names: ['Dust_Storm']
"""
with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(dataset_yaml_content)

def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Training on: {device.upper()}")

    model = YOLO('yolov8m.pt')

    model.train(
        data="/kaggle/working/dataset.yaml",
        epochs=200,
        batch=8,
        imgsz=960,
        device=device,
        workers=4,
        lr0=0.0006006927893233255,
        mosaic=0.416367919520838,
        mixup=0.14510771760161598,
        scale=0.27259964008817533,
        translate=0.08212054544651644,
        fliplr=0.491227741270323,
        hsv_h=0.015,
        hsv_s=0.6,
        hsv_v=0.4,
        perspective=0.001,
        flipud=0.05,
        close_mosaic=10,
        erasing=0.3,
        auto_augment=None,
        optimizer="AdamW",
        lrf=0.01,
        cos_lr=True,
        warmup_epochs=3,
        warmup_bias_lr=0.1,
        warmup_momentum=0.8,
        weight_decay=0.0005,
        dropout=0.1,
        freeze=0,
        patience=50,
        amp=True,
        seed=42,
        deterministic=True,
        val=True,
        plots=True,
        multi_scale=False,
        project="/kaggle/working/runs/detect",
        name="final_train_yolov8m"
    )

if __name__ == "__main__":
    main()


In [ ]:
from ultralytics import YOLO
import torch

dataset_yaml_content = """
train: /kaggle/input/mars-daily-images/dataset/images/train
val: /kaggle/input/mars-daily-images/dataset/images/val

nc: 1
names: ['Dust_storm']
"""
with open("/kaggle/working/dataset.yaml", "w") as f:
    f.write(dataset_yaml_content)

def resume_training():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Resuming training on: {device.upper()}")

    model = YOLO('/kaggle/input/lastcheckpoint/pytorch/default/1/last (1).pt')

    model.train(
        resume=True,
        data="/kaggle/working/dataset.yaml",
        device=device,
        project="/kaggle/working/runs/detect",
        name="final_train_yolov8m"
    )

if __name__ == "__main__":
    resume_training()
